# Project 4
### team members:
### 1. Sasi Kanduri &emsp; 2. Vikas Mishra &emsp;3. Ashish Thranath Kotian

## Base Model from project 3

In [178]:
import numpy as np
from numpy import array
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding,Activation, Dropout, LSTM, MultiHeadAttention
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, Attention
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model, image_dataset_from_directory
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, Rescaling, MaxPooling2D, concatenate

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report

### Importing training/validation/test Data from the files downlaoded with image_downloader.py

In [179]:
df_train = pd.read_csv('final_df_train.csv', index_col=False)
df_validate = pd.read_csv('final_df_validate.csv', index_col=False)
df_test = pd.read_csv('final_df_test.csv', index_col=False)

y_train = df_train['2_way_label'].to_numpy()
y_validate = df_validate['2_way_label'].to_numpy()
y_test = df_test['2_way_label'].to_numpy()

print(f"train set x shape : {df_train.shape} y shape: {y_train.shape}")
print(f"validate set x shape : {df_validate.shape} y shape: {y_validate.shape}")
print(f"test set x shape : {df_test.shape} y shape: {y_test.shape}")


train set x shape : (1921, 2) y shape: (1921,)
validate set x shape : (486, 2) y shape: (486,)
test set x shape : (477, 2) y shape: (477,)


### Text Model with GLoVe Embedding for text data

### Contractions

In [180]:
contractions = {
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how does",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
" u ": " you ",
" ur ": " your ",
" n ": " and "}

### Function to clean the text using the contractions

In [181]:
def get_clean_text(x):
    if type(x) is str:
        x = x.lower()
        for key in contractions:
            value = contractions[key]
            x = x.replace(key, value)
        return x
    else:
        return x


### Applying Contraction Function on Train/Validation/Testing Data

In [182]:
df_train['text with images'] = df_train['text with images'].apply(lambda x: get_clean_text(x))
df_validate['text with images'] = df_validate['text with images'].apply(lambda x: get_clean_text(x))
df_test['text with images'] = df_test['text with images'].apply(lambda x: get_clean_text(x))

### Train Data After Applying Contraction Function

In [183]:
df_train['text with images']

0       my walgreens offbrand mucinex was engraved wit...
1           hackers leak emails from uae ambassador to us
2                                puppy taking in the view
3       bride and groom exchange vows after fatal shoo...
4                                           major thermos
                              ...                        
1916              us solar installations hit million mark
1917    thought this little flower was pretty interesting
1918                          the john rylands library oc
1919    this snail who has climbed up to my first stor...
1920    plop your infant in front of these pictures of...
Name: text with images, Length: 1921, dtype: object

### Validation Data After Applying Contraction Function

In [184]:
df_validate['text with images']

0                   my xbox controller says hi
1               new image from the mandalorian
2                say hello to my little friend
3                   watch your step little one
4      this tree i found with a solo cup on it
                        ...                   
481                               high fashion
482                    years old world records
483                railroad track senior photo
484         a rare photograph of billy the kid
485        the onion reviews crazy rich asians
Name: text with images, Length: 486, dtype: object

### Test Data After Applying Contraction Function

In [185]:
df_test['text with images']

0                                              stargazer
1                                                   yeah
2      pd phoenix car thief gets instructions from yo...
3      as trump accuses iran he has one problem his o...
4                                    believers hezbollah
                             ...                        
472                                           angry baby
473                            this sign in a restaurant
474                                       disaster pratt
475    reading the manifesto of russia painting by gr...
476                             httpsiimgurcomxcvuzmtjpg
Name: text with images, Length: 477, dtype: object

### Generating Tokens using keras tokenizer

In [186]:
text_train = df_train['text with images'].tolist()
text_validate = df_validate['text with images'].tolist()
text_test = df_test['text with images'].tolist()

token = Tokenizer()

token.fit_on_texts(text_train)


### vocabulary size

In [187]:
vocab_size  = len(token.word_index) + 1
vocab_size

5063

### Encoding Text to Sequences for Train/Validation/Test Data

In [188]:
encoded_text_train = token.texts_to_sequences(text_train)
encoded_text_validate = token.texts_to_sequences(text_validate)
encoded_text_test = token.texts_to_sequences(text_test)

### Padding

In [189]:
max_length = 225
text_train = pad_sequences(encoded_text_train, maxlen=max_length, padding='post')
text_validate = pad_sequences(encoded_text_validate, maxlen=max_length, padding='post')
text_test = pad_sequences(encoded_text_test, maxlen=max_length, padding='post')

### Converting data to numpy arrays

In [190]:
text_train = np.array(text_train)
text_validate = np.array(text_validate)
text_test = np.array(text_test)

### Reading the Glove Vectors for each word from the pretrained embeddings file 'glove.twitter.27B.25d.txt' - 25 dimensions

In [191]:
glove_vectors = dict()

In [192]:
%%time

file = open('glove.twitter.27B.25d.txt', encoding='utf-8')

for line in file:
    values = line.split()
    word = values[0]
    vectors = np.asarray(values[1: ])
    glove_vectors[word] = vectors
file.close()

CPU times: user 4.66 s, sys: 207 ms, total: 4.86 s
Wall time: 4.86 s


In [193]:
len(glove_vectors)

1193514

### Creating Word Matrix For Each Word In Token

In [194]:
word_vector_matrix = np.zeros((vocab_size, 25))

tokens = []
labels = []

for word, index in token.word_index.items():   # index returned here starts with 1 so we need set vocab_size = len(token.word_index) + 1  to be able to index up to the greatest token ID
    vector = glove_vectors.get(word)
    if vector is not None:
        word_vector_matrix[index] = vector


### Word matrix size

In [195]:
word_vector_matrix.shape

(5063, 25)

### Building The Glove Embedding Model

In [196]:
# text model
input1 = Input(shape = (max_length))
embedding = Embedding(vocab_size, 25, weights = [word_vector_matrix], trainable = False, name='embedding')(input1)

layer1 = Conv1D(64, 8, activation = 'relu')(embedding)
layer2 = MaxPooling1D(2)(layer1)
layer3 = Conv1D(32, 4, activation = 'relu')(layer2)
layer4 = MaxPooling1D(2)(layer3)

flat1 = Flatten()(layer4)
dense1 = Dense(128, activation='relu')(flat1)

### CNN model for images

### Converting target label to list for labels

In [197]:

y_labels_train = y_train.tolist()
y_labels_validate = y_validate.tolist()
y_labels_test = y_test.tolist()

### Creating dataset for our train/validation/test Data

In [198]:

image_train = image_dataset_from_directory("images_train", labels=y_labels_train, label_mode="binary", image_size=(64,64), batch_size=32, color_mode='rgb')
image_val = image_dataset_from_directory("images_validate", labels=y_labels_validate, label_mode="binary", image_size=(64,64), batch_size=32, color_mode='rgb')
image_test = image_dataset_from_directory("images_test", labels=y_labels_test, label_mode="binary", image_size=(64,64), batch_size=32, color_mode='rgb')

Found 1921 files belonging to 2 classes.
Found 486 files belonging to 2 classes.
Found 477 files belonging to 2 classes.


### Autotune with keras for performance

In [199]:
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = image_train.prefetch(buffer_size=AUTOTUNE)

# train data
data_batches = []

for batch in train_dataset:
    data_batches.append(batch[0])

image_data_train = np.concatenate(data_batches, axis=0)

#validation data
val_dataset = image_val.prefetch(buffer_size=AUTOTUNE)

data_batches = []

for batch in val_dataset:
    try:
        data_batches.append(batch[0])
    except:
        print(batch)

image_data_validate = np.concatenate(data_batches, axis=0)

# testd data
test_dataset = image_test.prefetch(buffer_size=AUTOTUNE)

data_batches = []

for batch in test_dataset:
    data_batches.append(batch[0])

image_data_test = np.concatenate(data_batches, axis=0)

In [200]:
print(image_data_train.shape)
print(image_data_validate.shape)
print(image_data_test.shape)

(1921, 64, 64, 3)
(486, 64, 64, 3)
(477, 64, 64, 3)


### Building the CNN Model

In [201]:
input2 = Input(shape=(64,64,3))
rescaling = Rescaling(1./255)(input2) # scale pixels

In [202]:
input2 = Input(shape=(64,64,3))
rescaling = Rescaling(1./255)(input2) # scale pixels

conv1 = Conv2D(100, (4, 4), activation='relu')(rescaling)
pool1 = MaxPooling2D((2, 2), padding='same')(conv1)
conv2 = Conv2D(64, (2, 2), activation='relu')(pool1)
pool2 = MaxPooling2D((2, 2), padding='same')(conv2)

flat_layer = Flatten()(pool2)
dense2 = Dense(64, activation='relu')(flat_layer)

image_model = Model(inputs = input2, outputs = dense2)

### Creating a multimodal architecture by merging the dense layers of text and image models

In [203]:
merge = concatenate([dense1, dense2])

hidden1 = Dense(128, activation='relu')(merge)
hidden2 = Dense(32, activation='relu')(hidden1)
output = Dense(1, activation='sigmoid')(hidden2)

model = Model(inputs=[input1, input2], outputs=output)

model.summary()

Model: "model_20"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_13 (InputLayer)       [(None, 225)]                0         []                            
                                                                                                  
 input_15 (InputLayer)       [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 embedding (Embedding)       (None, 225, 25)              126575    ['input_13[0][0]']            
                                                                                                  
 rescaling_9 (Rescaling)     (None, 64, 64, 3)            0         ['input_15[0][0]']            
                                                                                           

### Defining ModelCheckPoint & EarlyStopping

In [204]:
checkpointer = ModelCheckpoint(filepath="best_nultimodal.hdf5", verbose=2, save_best_only=True, monitor='val_loss')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=2, mode='auto')

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

model.fit([text_train, image_data_train], y_train, epochs=50, callbacks=[monitor, checkpointer], batch_size = 32,
            validation_data=([text_validate, image_data_validate], y_validate))

Epoch 1/50
61/61 [==============================] - ETA: 0s - loss: 0.6954 - accuracy: 0.5997
Epoch 1: val_loss improved from inf to 0.68312, saving model to best_nultimodal.hdf5
61/61 [==============================] - 16s 162ms/step - loss: 0.6954 - accuracy: 0.5997 - val_loss: 0.6831 - val_accuracy: 0.6317
Epoch 2/50
 5/61 [=>............................] - ETA: 1s - loss: 0.6304 - accuracy: 0.7000

/Users/shashi/Documents/219-ML/mlenv/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


61/61 [==============================] - ETA: 0s - loss: 0.5807 - accuracy: 0.6965
Epoch 2: val_loss improved from 0.68312 to 0.61762, saving model to best_nultimodal.hdf5
61/61 [==============================] - 2s 27ms/step - loss: 0.5807 - accuracy: 0.6965 - val_loss: 0.6176 - val_accuracy: 0.6667
Epoch 3/50
61/61 [==============================] - ETA: 0s - loss: 0.5759 - accuracy: 0.6955
Epoch 3: val_loss did not improve from 0.61762
61/61 [==============================] - 1s 24ms/step - loss: 0.5759 - accuracy: 0.6955 - val_loss: 0.6422 - val_accuracy: 0.6831
Epoch 4/50
61/61 [==============================] - ETA: 0s - loss: 0.5857 - accuracy: 0.7054
Epoch 4: val_loss did not improve from 0.61762
61/61 [==============================] - 2s 26ms/step - loss: 0.5857 - accuracy: 0.7054 - val_loss: 0.7173 - val_accuracy: 0.6523
Epoch 5/50
61/61 [==============================] - ETA: 0s - loss: 0.8048 - accuracy: 0.6486
Epoch 5: val_loss did not improve from 0.61762
61/61 [========

### Predicting with test data

In [205]:
model.load_weights("best_nultimodal.hdf5")
pred = model.predict([text_test, image_data_test])

pred = (pred > 0.5)

15/15 [==============================] - 2s 77ms/step


### Results

In [206]:
base_mode_result = classification_report(y_test, pred)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.75      0.72      0.73       278
           1       0.63      0.66      0.64       199

    accuracy                           0.69       477
   macro avg       0.69      0.69      0.69       477
weighted avg       0.70      0.69      0.70       477



# Attention Mechanism

### Building LSTM for text model and reshaping the dimensions of visual features for attention layers

In [207]:
# image model output
conv_output_reshaped = tf.keras.layers.Reshape((-1, 64), name='image_model')(pool2)

# text model output
lstm_layer1 = LSTM(units=100, return_sequences=True)(embedding)
lstm_layer2 = LSTM(units=64, return_sequences=True, name="for_attention")(lstm_layer1)

## Building cross attention layer model

In [208]:

cross_attention_layer = MultiHeadAttention(key_dim=64, num_heads=8)(conv_output_reshaped, lstm_layer2)

add_norm = layers.LayerNormalization(epsilon=1e-6)(cross_attention_layer + lstm_layer2)

pool = layers.GlobalAveragePooling1D()(add_norm)

out1 = layers.LayerNormalization(epsilon=1e-6)(pool)

ffn1 = Dense(64, activation='relu')(out1)
ffn2 = Dense(32, activation='relu')(ffn1)

final = Dense(1, activation='sigmoid')(ffn2)


cross_model = Model(inputs = [input1, input2], outputs = final)
cross_model.summary()

Model: "model_21"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_15 (InputLayer)       [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 rescaling_9 (Rescaling)     (None, 64, 64, 3)            0         ['input_15[0][0]']            
                                                                                                  
 conv2d_8 (Conv2D)           (None, 61, 61, 100)          4900      ['rescaling_9[0][0]']         
                                                                                                  
 max_pooling2d_8 (MaxPoolin  (None, 31, 31, 100)          0         ['conv2d_8[0][0]']            
 g2D)                                                                                      

In [209]:
# checkpointer = ModelCheckpoint(filepath="cross_attention_model.hdf5", verbose=2, save_best_only=True, monitor='val_loss')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=2, mode='auto')

cross_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

cross_model.fit([text_train, image_data_train], y_train, epochs=50, callbacks=[monitor],
            validation_data=([text_validate, image_data_validate], y_validate))

Epoch 1/50
61/61 [==============================] - ETA: 0s - loss: 0.7740 - accuracy: 0.5846
Epoch 1: val_loss improved from inf to 0.66058, saving model to cross_attention_model.hdf5
61/61 [==============================] - 50s 549ms/step - loss: 0.7740 - accuracy: 0.5846 - val_loss: 0.6606 - val_accuracy: 0.6193
Epoch 2/50


/Users/shashi/Documents/219-ML/mlenv/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


61/61 [==============================] - ETA: 0s - loss: 0.6949 - accuracy: 0.5471
Epoch 2: val_loss did not improve from 0.66058
61/61 [==============================] - 15s 246ms/step - loss: 0.6949 - accuracy: 0.5471 - val_loss: 0.6923 - val_accuracy: 0.5988
Epoch 3/50
61/61 [==============================] - ETA: 0s - loss: 0.6899 - accuracy: 0.5711
Epoch 3: val_loss did not improve from 0.66058
61/61 [==============================] - 15s 241ms/step - loss: 0.6899 - accuracy: 0.5711 - val_loss: 0.6689 - val_accuracy: 0.5988
Epoch 4/50
61/61 [==============================] - ETA: 0s - loss: 0.6831 - accuracy: 0.5679
Epoch 4: val_loss improved from 0.66058 to 0.65361, saving model to cross_attention_model.hdf5
61/61 [==============================] - 15s 242ms/step - loss: 0.6831 - accuracy: 0.5679 - val_loss: 0.6536 - val_accuracy: 0.5988
Epoch 5/50
61/61 [==============================] - ETA: 0s - loss: 0.6227 - accuracy: 0.6158
Epoch 5: val_loss improved from 0.65361 to 0.63092

In [210]:
pred = cross_model.predict([text_test, image_data_test])

pred = (pred > 0.5)

15/15 [==============================] - 8s 280ms/step


### Results

In [211]:
cross_model_results = classification_report(y_test, pred)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.58      1.00      0.74       278
           1       0.00      0.00      0.00       199

    accuracy                           0.58       477
   macro avg       0.29      0.50      0.37       477
weighted avg       0.34      0.58      0.43       477



/Users/shashi/Documents/219-ML/mlenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shashi/Documents/219-ML/mlenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shashi/Documents/219-ML/mlenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

## Building self attention layer

In [212]:
combined_features = concatenate([conv_output_reshaped, lstm_layer2], name="merged")

In [213]:
self_attention = MultiHeadAttention(key_dim=64, num_heads=8)(combined_features, combined_features)

add_norm = layers.LayerNormalization(epsilon=1e-6)(self_attention + combined_features)

pool = layers.GlobalAveragePooling1D()(add_norm)

out1 = layers.LayerNormalization(epsilon=1e-6)(pool)

ffn1 = Dense(64, activation='relu')(out1)
ffn2 = Dense(32, activation='relu')(ffn1)

final = Dense(1, activation='sigmoid')(ffn2)

self_attention_model = Model(inputs = [input1, input2], outputs = final)
self_attention_model.summary()

Model: "model_22"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_15 (InputLayer)       [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 rescaling_9 (Rescaling)     (None, 64, 64, 3)            0         ['input_15[0][0]']            
                                                                                                  
 conv2d_8 (Conv2D)           (None, 61, 61, 100)          4900      ['rescaling_9[0][0]']         
                                                                                                  
 max_pooling2d_8 (MaxPoolin  (None, 31, 31, 100)          0         ['conv2d_8[0][0]']            
 g2D)                                                                                      

In [214]:
# checkpointer = ModelCheckpoint(filepath="self_attention_model.hdf5", verbose=2, save_best_only=True, monitor='val_loss')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=2, mode='auto')

self_attention_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

self_attention_model.fit([text_train, image_data_train], y_train, epochs=50, callbacks=[monitor],
            validation_data=([text_validate, image_data_validate], y_validate))

Epoch 1/50
61/61 [==============================] - ETA: 0s - loss: 0.6491 - accuracy: 0.6330
Epoch 1: val_loss improved from inf to 0.64728, saving model to self_attention_model.hdf5
61/61 [==============================] - 57s 611ms/step - loss: 0.6491 - accuracy: 0.6330 - val_loss: 0.6473 - val_accuracy: 0.5967
Epoch 2/50


/Users/shashi/Documents/219-ML/mlenv/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


61/61 [==============================] - ETA: 0s - loss: 0.6117 - accuracy: 0.5945
Epoch 2: val_loss did not improve from 0.64728
61/61 [==============================] - 16s 264ms/step - loss: 0.6117 - accuracy: 0.5945 - val_loss: 0.6550 - val_accuracy: 0.5638
Epoch 3/50
61/61 [==============================] - ETA: 0s - loss: 0.6641 - accuracy: 0.5632
Epoch 3: val_loss did not improve from 0.64728
61/61 [==============================] - 16s 257ms/step - loss: 0.6641 - accuracy: 0.5632 - val_loss: 0.6711 - val_accuracy: 0.5988
Epoch 4/50
61/61 [==============================] - ETA: 0s - loss: 0.7028 - accuracy: 0.5549
Epoch 4: val_loss did not improve from 0.64728
61/61 [==============================] - 15s 253ms/step - loss: 0.7028 - accuracy: 0.5549 - val_loss: 0.7027 - val_accuracy: 0.5926
Epoch 5/50
61/61 [==============================] - ETA: 0s - loss: 0.7110 - accuracy: 0.5377
Epoch 5: val_loss did not improve from 0.64728
61/61 [==============================] - 16s 255ms/

In [215]:
# self_attention_model.load_weights("self_attention_model.hdf5")
pred = self_attention_model.predict([text_test, image_data_test])

pred = (pred > 0.5)

15/15 [==============================] - 8s 293ms/step


In [216]:
self_attention_model_results = classification_report(y_test, pred)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.59      0.99      0.73       278
           1       0.56      0.03      0.05       199

    accuracy                           0.58       477
   macro avg       0.57      0.51      0.39       477
weighted avg       0.57      0.58      0.45       477



## Building co-attention layer model

In [217]:
co_attention_layer1 = MultiHeadAttention(key_dim=64, num_heads=8)(conv_output_reshaped, lstm_layer2)
co_attention_layer2 = MultiHeadAttention(key_dim=64, num_heads=8)(lstm_layer2, conv_output_reshaped)

fused = concatenate([co_attention_layer1, co_attention_layer2])

add_norm = layers.LayerNormalization(epsilon=1e-6)(fused + concatenate([conv_output_reshaped, lstm_layer2]))

pool = layers.GlobalAveragePooling1D()(add_norm)

out1 = layers.LayerNormalization(epsilon=1e-6)(pool)

ffn1 = Dense(64, activation='relu')(out1)
ffn2 = Dense(32, activation='relu')(ffn1)

final = Dense(1, activation='sigmoid')(ffn2)

co_attention_model = Model(inputs = [input1, input2], outputs = final)
co_attention_model.summary()

Model: "model_23"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_15 (InputLayer)       [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 rescaling_9 (Rescaling)     (None, 64, 64, 3)            0         ['input_15[0][0]']            
                                                                                                  
 conv2d_8 (Conv2D)           (None, 61, 61, 100)          4900      ['rescaling_9[0][0]']         
                                                                                                  
 max_pooling2d_8 (MaxPoolin  (None, 31, 31, 100)          0         ['conv2d_8[0][0]']            
 g2D)                                                                                      

In [218]:
# checkpointer = ModelCheckpoint(filepath="co_attention_model.hdf5", verbose=2, save_best_only=True, monitor='val_loss')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=2, mode='auto')

co_attention_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

co_attention_model.fit([text_train, image_data_train], y_train, epochs=50, callbacks=[monitor],
            validation_data=([text_validate, image_data_validate], y_validate))

Epoch 1/50
61/61 [==============================] - ETA: 0s - loss: 0.6419 - accuracy: 0.6143
Epoch 1: val_loss improved from inf to 0.70117, saving model to co_attention_model.hdf5
61/61 [==============================] - 66s 752ms/step - loss: 0.6419 - accuracy: 0.6143 - val_loss: 0.7012 - val_accuracy: 0.6276
Epoch 2/50


/Users/shashi/Documents/219-ML/mlenv/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


41/61 [===================>..........] - ETA: 5s - loss: 0.6004 - accuracy: 0.6265

KeyboardInterrupt: 

In [ ]:
# co_attention_model.load_weights("co_attention_model.hdf5")
pred = co_attention_model.predict([text_test, image_data_test])

pred = (pred > 0.5)

In [ ]:
co_attention_model_results = classification_report(y_test, pred)
print(classification_report(y_test, pred))

In [ ]:
from sklearn.metrics import classification_report
from prettytable import PrettyTable

print("================================= No Attention ======================================")
print(base_mode_result)
print("================================= Cross Attention ====================================")
print(cross_model_results)
print("================================= Self Attention =====================================")
print(self_attention_model_results)
print("================================= Co Attention  ======================================")
print(co_attention_model_results)


## VGG

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

# Loading VGG16 model

base_model = VGG16(weights="imagenet", include_top=False, input_tensor=input2)
base_model.trainable = False

vgg_model = Sequential()

for x in base_model.layers[:4]:
    vgg_model.add(x)

vgg_last = vgg_model.layers[-1].output
conv1 = Conv2D(100, (4, 4), activation='relu')(vgg_last)
pool1 = MaxPooling2D((2, 2), padding='same')(conv1)
conv2 = Conv2D(64, (2, 2), activation='relu')(pool1)
pool2 = MaxPooling2D((2, 2), padding='same')(conv2)

vgg_output_reshaped = tf.keras.layers.Reshape((-1, 64), name='layer_taken_out_for_attentions')(pool2)

sample_model = Model(inputs = vgg_model.layers[0].output, outputs=vgg_output_reshaped)

sample_model.summary()

## Building cross attention layer model

In [ ]:

cross_attention_layer = MultiHeadAttention(key_dim=64, num_heads=8)(vgg_output_reshaped, lstm_layer2)

pool = layers.GlobalAveragePooling1D()(cross_attention_layer)

out1 = layers.LayerNormalization(epsilon=1e-6)(pool)

ffn_output = Sequential(
            [layers.Dense(64, activation="relu"), layers.Dense(32)]
        )(out1)

out2 = layers.LayerNormalization(epsilon=1e-6)(ffn_output)

final = Dense(1, activation='sigmoid')(out2)

cross_model = Model(inputs = [input1, input2], outputs = final)
cross_model.summary()

In [ ]:
checkpointer = ModelCheckpoint(filepath="cross_attention_model_vgg.hdf5", verbose=2, save_best_only=True, monitor='val_loss')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=2, mode='auto')

cross_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

cross_model.fit([text_train, image_data_train], y_train, epochs=50, callbacks=[monitor, checkpointer],
            validation_data=([text_validate, image_data_validate], y_validate))

In [ ]:
cross_model.load_weights("cross_attention_model_vgg.hdf5")
pred = cross_model.predict([text_test, image_data_test])

pred = (pred > 0.5)

### Results

In [ ]:
print(classification_report(y_test, pred))